The example is based on a dataset that is publicly available from the UCI Machine Learning Repository (Asuncion and Newman, 2007)[http://mlearn.ics.uci.edu/MLRepository.html]. 
The dataset consists of several hundred human cell sample records, each of which contains the values of a set of cell characteristics. The fields in each record are:

Field name	Description--
ID	Clump thickness
Clump	Clump thickness
UnifSize	Uniformity of cell size
UnifShape	Uniformity of cell shape
MargAdh	Marginal adhesion
SingEpiSize	Single epithelial cell size
BareNuc	Bare nuclei
BlandChrom	Bland chromatin
NormNucl	Normal nucleoli
Mit	Mitoses
Class	Benign or malignant [The Class field contains the diagnosis, as confirmed by separate medical procedures, as to whether the samples are benign (value = 2) or malignant (value = 4).]

Objective- Use Support Vector Machine(SVM) to predict the class ( Benign or Malignant) for future data.


In [8]:
!wget -O cell_samples.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv


--2018-10-14 15:08:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/cell_samples.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20675 (20K) [text/csv]
Saving to: ‘cell_samples.csv’

cell_samples.csv    100%[=====================>]  20.19K  --.-KB/s   in 0.02s  

2018-10-14 15:08:49 (940 KB/s) - ‘cell_samples.csv’ saved [20675/20675]



In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

General libraries for creating data-frames using the data in the CSV file:

In [10]:
df = pd.read_csv('cell_samples.csv')
df.head(5)

ID  Clump  UnifSize  UnifShape  MargAdh  SingEpiSize BareNuc  \
0  1000025      5         1          1        1            2       1   
1  1002945      5         4          4        5            7      10   
2  1015425      3         1          1        1            2       2   
3  1016277      6         8          8        1            3       4   
4  1017023      4         1          1        3            2       1   

   BlandChrom  NormNucl  Mit  Class  
0           3         1    1      2  
1           3         2    1      2  
2           3         1    1      2  
3           3         7    1      2  
4           3         1    1      2

Preprocessing data for data-type consistency: 

In [20]:
print(df.dtypes)
df = df[pd.to_numeric(df['BareNuc'], errors='coerce').notnull()]
df['BareNuc'] = df['BareNuc'].astype(int)
print('\nTransformed into :\n', df.dtypes )


ID             int64
Clump          int64
UnifSize       int64
UnifShape      int64
MargAdh        int64
SingEpiSize    int64
BareNuc        int64
BlandChrom     int64
NormNucl       int64
Mit            int64
Class          int64
dtype: object

Transformed into :
 ID             int64
Clump          int64
UnifSize       int64
UnifShape      int64
MargAdh        int64
SingEpiSize    int64
BareNuc        int64
BlandChrom     int64
NormNucl       int64
Mit            int64
Class          int64
dtype: object


Use 'df.columns' to grab features and insert values into array X
and insert Class(2==benign and 4==malignant) into array Y :


In [27]:
df.columns #['ID', 'Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize','BareNuc', 'BlandChrom', 'NormNucl', 'Mit', 'Class']
X = np.asarray(df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize',
       'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']])
Y = np.asarray(df['Class'])
print('Sample of X =\n',X[0:5])
print('Sample of Y =\n',Y[0:5])

Sample of X =
 [[ 5  1  1  1  2  1  3  1  1]
 [ 5  4  4  5  7 10  3  2  1]
 [ 3  1  1  1  2  2  3  1  1]
 [ 6  8  8  1  3  4  3  7  1]
 [ 4  1  1  3  2  1  3  1  1]]
Sample of Y =
 [2 2 2 2 2]


Use 'train_test_split' to randomly separate data into train(X,Y) and test(X,Y) and check shape for any inconsistencies:

In [84]:
from sklearn.model_selection import train_test_split
train_X , test_X , train_Y , test_Y = train_test_split(X,Y,test_size=.2,random_state=4)
print('Train = 80% of all rows and 9 columns :', train_X.shape ,' ', train_Y.shape)
print(' Test = 20% of all rows and 9 columns :', test_X.shape ,' ', test_Y.shape)
print(train_X[:,0].shape, ' + ',test_X[:,0].shape,' =  {0} total rows'.format(546+137))

Train = 80% of all rows and 9 columns : (546, 9)   (546,)
 Test = 20% of all rows and 9 columns : (137, 9)   (137,)
(546,)  +  (137,)  =  683 total rows


Create the SVM model and fit it with the training data, then predict testing data. Before testing every kernel type use a loop to find the optimized polynomial kernel degree by comparing jaccard_similarity_scores for different polynomials:

In [126]:
from sklearn.svm import SVC
from sklearn.metrics import jaccard_similarity_score
deg = {1:'',2:'',3:'',4:'',5:'',6:'',7:'',8:'',9:''}
for i,k in deg.items():
    clf = SVC(kernel='poly', degree=i)
    clf.fit(train_X , train_Y)
    Y_ = clf.predict(test_X)
    k = jaccard_similarity_score(test_Y,Y_)
    deg[i] = k
Max = max(deg , key=deg.get)
print(Max,deg[Max])


5 0.9781021897810219


Now that the optimized polynomial is found , compare it to the other simple vector machine kernel models
then use confusion_matrix to find true positive & true negative and false positive and false negative:

In [143]:
from sklearn.metrics import confusion_matrix 
kern = ['linear' , 'rbf' ,'sigmoid','poly']
for i in kern:
    clf=SVC(kernel=i , degree=5) if i=='poly' else SVC(kernel=i)
    clf.fit(train_X,train_Y)
    Y_= clf.predict(test_X)
    print(i , ' ', jaccard_similarity_score(test_Y,Y_))
    print(confusion_matrix(test_Y,Y_,labels=[4,2]))
    print('----------------------------')

linear   0.9635036496350365
[[47  0]
 [ 5 85]]
----------------------------
rbf   0.9635036496350365
[[47  0]
 [ 5 85]]
----------------------------
sigmoid   0.3357664233576642
[[ 0 47]
 [44 46]]
----------------------------
poly   0.9781021897810219
[[47  0]
 [ 3 87]]
----------------------------


The best model to use is Simple Vector Machine with a polynomial kernel to the 5th degree. 